In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=de871703736308fd91e192fb82fed0da6b467d51fb761f33c610afa68d4bfc0e
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import re 

In [5]:
import numpy as np

In [6]:
from numpy import save

In [7]:
file1 = open('input_bert.txt', 'r')
result_doc = file1.readlines()

filenames = []
find = re.compile(r"\w+(?=\s*-[^/])")
# Strips the newline character

for l in result_doc: 
  filenames.append((re.findall(find, l)[0]))

regex = re.compile(r'<DOC>(.*?)</DOC>', flags=re.DOTALL)
re_text = re.compile(r'<TEXT>(.*?)</TEXT>', flags=re.DOTALL)
re_docno = re.compile(r'<DOCNO>(.*?)</DOCNO>')

for filename in filenames: 

  writeto = filename + ".npy"
  f = open(filename, "r")
  text_doc = f.read()

  docs = []
  find = re.compile(r"\w+(?=\s*-[^/])")
  for l in result_doc: 
    docs.append((re.findall(find, l)))
  
  mydict = {}

    #the list of all the files we need to look at
  all_docs_matched = re.findall(regex, text_doc)
    #<-------- get all the text in the doc, text[x] = the text portion of one document--->
  text = []
  for one_doc in all_docs_matched:
    one_doc.replace('\n', '')
    text_one_doc = re.findall(re_text, one_doc)
    text.append(text_one_doc)
    #<------------------------------------------>

  doc_no = re.findall(re_docno, text_doc)#all the document numbers
  for x in range(len(doc_no)):
    if (doc_no[x].strip() in (' '.join(result_doc))):
    mydict[doc_no[x].strip()]=text[x][0]


  arr = list(mydict.values())

  sentence_embeddings = model.encode(arr)
  sentence_embeddings.shape
  save(writeto, sentence_embeddings)

KeyboardInterrupt: ignored

In [ ]:
file2 = open('cleaned_queries.txt', 'r')
query_doc = file2.readlines()
sentence_embeddings = model.encode(query_doc)
sentence_embeddings.shape
save("query_embed.npy", sentence_embeddings)


In [ ]:
from numpy import load

file1 = open('input_bert.txt', 'r')
result_doc = file1.readlines()
q_embed = load("query_embed.npy").tolist()

docs = []
find = re.compile(r"\w+(?=\s*-[^/])")
# Strips the newline character

for l in result_doc: 
  docs.append((re.findall(find, l)[0]))

#has all the names of the files we need to open
set_docs = set(docs)

re_docno = re.compile(r'<DOCNO>(.*?)</DOCNO>')

mydict = {}
# key -> doc name, value -> text embedding 

for doc in set_docs:
  docname = "docs/"+ doc
  f = open(docname, "r")
  text_doc = f.readlines()
  # docs = load("embeds/AP880212.npy")
  filename = "embeds/"+doc + ".npy"
  embed = load(filename)
  
  doc_no = re.findall(re_docno, text_doc)#all the document numbers 

  for x in range(len(doc_no)):
    # for every document number, add to the dictionary if that document is a ranked file
    if (doc_no[x].strip() in (' '.join(result_doc))):
      mydict[doc_no[x].strip()]=embed[x]


#q_dict is a dictionary where
# key-> query number, value-> the 1000 ranked doc names as a list
q_dict = {}
for i in range(len(q_embed)):
  my_int = i*1000
  tmp = result_doc[my_int:(my_int+1000)]
  q_dict[i] = tmp

file_write_list = []
for key, value in q_dict.items():
  arr = []
  # key is query number
  # value is a list of doc names 
  # AP880815-0061 \n'
  for docname in value:
    arr.append(mydict[docname.strip()]) #create an array with the embeddings

  my_arr = cosine_similarity([q_embed[0]], arr[0:])[0].tolist()
  for i in range(1000):
    print(q_dict[0][i] + ":")
    print(my_arr[i])
    print("\n")
    list1, list2 = zip(*sorted(zip(my_arr, q_dict[i]), reverse=True))
    for num in range(len(list1)):
        # query number, q0, doc name, rank, cosine sim 
      # print(str(i) + " Q0 "+ list2[num].strip()+" "+ str(num+1)+" "+ str(list1[num]) + "my_search \n")
      file_write_list.append(str(key) + " Q0 "+ list2[num].strip()+" "+ str(num+1)+" "+ str(list1[num]) + " my_search \n")

  with open('bert_results.txt', 'w') as f:
    for line in file_write_list:
      f.write(line)
